In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch.optim as optim
from torchsummary import summary
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
import math
import os

#### 1- Define Data Transformations

In [7]:
# Define transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Mean and std for ImageNet
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

#### 2- Download and load the Dataset

In [4]:
import os
import zipfile
import requests
from tqdm import tqdm

# Function to download a file from a URL
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    total_size = int(r.headers.get('content-length', 0))
    with open(save_path, 'wb') as fd:
        for chunk in tqdm(r.iter_content(chunk_size=chunk_size), total=total_size//chunk_size, unit='KB', unit_scale=True):
            fd.write(chunk)

# URL and path for the Hymenoptera dataset
url = 'https://download.pytorch.org/tutorial/hymenoptera_data.zip'
data_dir = 'data'
zip_path = os.path.join(data_dir, 'hymenoptera_data.zip')

# Create directory if it does not exist
os.makedirs(data_dir, exist_ok=True)

# Download the dataset
download_url(url, zip_path)

# Extract the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)

print("Dataset downloaded and extracted!")


369kKB [00:12, 28.6kKB/s]                           


Dataset downloaded and extracted!


In [10]:
# Load datasets
data_dir = 'data/hymenoptera_data'  # Example dataset directory
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(dataset_sizes)
print(class_names)

{'train': 244, 'val': 153}
['ants', 'bees']


#### 3- Load a Pre-trained Model

In [14]:
# Load pre-trained model
model_ft = models.resnet18(pretrained=True)

# Modify the final layer
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device) # stands for model fine-tuned

#### 4- Define the Loss Function and Optimizer

In [15]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

#### 5- Train the Model

In [16]:
import time
import copy

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())     # save the best model weights
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):   # Enables gradient computation only during training.
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6188 Acc: 0.7705
val Loss: 81.4885 Acc: 0.5425

Epoch 1/24
----------
train Loss: 0.5858 Acc: 0.7828
val Loss: 34.3561 Acc: 0.5490

Epoch 2/24
----------
train Loss: 0.5317 Acc: 0.7500
val Loss: 2.3087 Acc: 0.6601

Epoch 3/24
----------
train Loss: 0.4562 Acc: 0.8074
val Loss: 8.9824 Acc: 0.5752

Epoch 4/24
----------
train Loss: 0.3977 Acc: 0.8115
val Loss: 0.5048 Acc: 0.7712

Epoch 5/24
----------
train Loss: 0.3071 Acc: 0.8443
val Loss: 0.5140 Acc: 0.8431

Epoch 6/24
----------
train Loss: 0.2916 Acc: 0.8934
val Loss: 0.4730 Acc: 0.8758

Epoch 7/24
----------
train Loss: 0.2585 Acc: 0.8689
val Loss: 0.3885 Acc: 0.8954

Epoch 8/24
----------
train Loss: 0.2444 Acc: 0.8934
val Loss: 0.3156 Acc: 0.8824

Epoch 9/24
----------
train Loss: 0.1821 Acc: 0.9303
val Loss: 0.2907 Acc: 0.9085

Epoch 10/24
----------
train Loss: 0.2135 Acc: 0.9098
val Loss: 0.2642 Acc: 0.9085

Epoch 11/24
----------
train Loss: 0.1859 Acc: 0.9098
val Loss: 0.2604 Acc: 0.9150



#### 6- Evaluate the Model

In [17]:
model_ft.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in dataloaders['val']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the validation images: {100 * correct / total:.2f}%')

Accuracy of the model on the validation images: 91.50%


# Saving and Loading

#### 1- Saving and Loading the Entire Model

In [18]:
torch.save(model_ft, 'model_ft.pth')

In [19]:
model = torch.load('model_ft.pth')

#### 2- Saving and Loading Only the Model State Dict (Recommended)

In [20]:
torch.save(model_ft.state_dict(), 'model_ft_state_dict.pth')

In [21]:
# Load pre-trained model
model_ft = models.resnet18(pretrained=True)

# Modify the final layer
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device) # stands for model fine-tuned

model_ft.load_state_dict(torch.load('model_ft_state_dict.pth'))
model.eval()  # Set the model to evaluation mode

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

#### 3- Saving and Loading the Checkpoint

In [ ]:
checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
}
torch.save(checkpoint, 'checkpoint.pth')

In [ ]:
model = ModelClass()
optimizer = OptimizerClass(model.parameters())

checkpoint = torch.load('checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']